In [1]:
from sqlalchemy.ext.asyncio import async_sessionmaker, create_async_engine
import sqlalchemy as sqla
from config import env


engine = create_async_engine(
    sqla.URL.create(
        host=env.DB_URL,
        drivername=env.DB_DRIVER,
        username=env.DB_USERNAME,
        password=env.DB_PASSWORD,
        # port=env.DB_PORT,
        port=5431,
        database=env.DB_NAME,
    ),
    pool_pre_ping=True,
)

ss = async_sessionmaker(
    engine,
    expire_on_commit=False,
)()


In [2]:
from db.postgresql.models.product import (
    Product,
    ProductPriceHistory,
    ProductStockHistory,
)
from tqdm.autonotebook import tqdm
from datetime import datetime
import datetime as dt

/tmp/ipykernel_25446/2328573263.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
MAX_PRICE_CHANGE_AMOUNT = 30
MIN_PRICE_CHANGE_AMOUNT = 5

MAX_STOCK_CHANGE_AMOUNT = 30
MIN_STOCK_CHANGE_AMOUNT = 5

MAX_ITEM_PER_ORDER = 10
MIN_ITEM_PER_ORDER = 3

MAX_AMOUNT_OF_ITEM_IN_ORDER = 10
MIN_AMOUNT_OF_ITEM_IN_ORDER = 10

MAX_PRICE = 40
MIN_PRICE = 5

MAX_SALE = 30
MIN_SALE = 0

MAX_PRICE_CHANGE_INTERVAL_DAY = 70
MIN_PRICE_CHANGE_INTERVAL_DAY = 20


MAX_RESTOCK_INTERVAL = 70
MIN_RESTOCK_INTERVAL = 20

In [4]:
import random

In [24]:
async with ss.begin():
    product_amount = await ss.scalar(sqla.select(sqla.func.count(Product.id))) or 0
    product_ids = (await ss.scalars(sqla.select(Product.id))).all()

In [25]:
async with ss.begin():
    pbar_prod = tqdm(total=product_amount - 1)
    pbar_sub = tqdm()

    current_day = datetime.now()

    for prod in product_ids:
        pbar_prod.set_description(f"Product : {prod}")

        price_amount = random.randint(
            MIN_PRICE_CHANGE_AMOUNT,
            MAX_PRICE_CHANGE_AMOUNT,
        )
        stock_amount = random.randint(
            MIN_STOCK_CHANGE_AMOUNT,
            MAX_STOCK_CHANGE_AMOUNT,
        )

        pbar_sub.reset(price_amount)
        pbar_sub.set_description("Generate price")

        prev_date = current_day
        for _ in range(price_amount):
            price = round(random.uniform(MIN_PRICE, MAX_PRICE), 2)
            sale_percent = round(random.uniform(MIN_SALE, MAX_SALE), 2)

            day_offset = dt.timedelta(
                days=random.randint(
                    MIN_PRICE_CHANGE_INTERVAL_DAY, MAX_PRICE_CHANGE_INTERVAL_DAY
                )
            )

            prev_date -= day_offset

            price_history = ProductPriceHistory(
                product_id=prod,
                date=prev_date,
                price=price,
                sale_percent=sale_percent,
            )

            ss.add(price_history)

            pbar_sub.update()

        pbar_sub.reset(stock_amount)
        pbar_sub.set_description("Generate stock")
        prev_date = current_day
        for _ in range(stock_amount):
            price = round(random.uniform(MIN_PRICE, MAX_PRICE) * 0.7, 2)
            stock = random.randint(
                MIN_STOCK_CHANGE_AMOUNT,
                MAX_STOCK_CHANGE_AMOUNT,
            )
            day_offset = dt.timedelta(
                days=random.randint(MIN_RESTOCK_INTERVAL, MAX_RESTOCK_INTERVAL)
            )

            prev_date -= day_offset
            stock = ProductStockHistory(
                product_id=prod,
                in_stock=stock,
                in_price=price,
                date=prev_date,
            )

            ss.add(stock)

            pbar_sub.update()

        pbar_prod.update()

    await ss.flush()

  0%|          | 0/22 [00:00<?, ?it/s]

0it [00:00, ?it/s]